In [3]:
import os
import re

def process_text(text):
    """
    Xử lý nội dung văn bản để phân tách và gán các cột theo định dạng token POS NER (BIO).
    """
    tokens = re.split(r'(\s+|<ENAMEX.*?>|</ENAMEX>)', text)
    tag_stack = []  # Stack lưu trữ các nhãn hiện tại
    result = []

    for token in tokens:
        token = token.strip()
        if not token:  # Bỏ qua token trống
            continue

        if token.startswith('<ENAMEX TYPE='):
            # Bắt đầu thẻ ENAMEX -> Lấy nhãn và đẩy vào stack
            tag = re.search(r'TYPE="([^"]+)"', token).group(1)
            tag_stack.append(tag)
        elif token == '</ENAMEX>':
            # Kết thúc thẻ ENAMEX -> Xóa nhãn khỏi stack
            if tag_stack:
                tag_stack.pop()
        else:
            # Tạo dòng dữ liệu với token và nhãn
            if tag_stack:
                current_tag = f'B-{tag_stack[-1]}' if len(tag_stack) == 1 else f'I-{tag_stack[-1]}'
            else:
                current_tag = 'O'
            result.append([token, current_tag])  # POS tag mặc định là 'O'

    return result

def process_folder(input_dir, output_file):
    """
    Xử lý tất cả file .muc trong thư mục và ghi kết quả vào file .txt.
    """
    with open(output_file, 'w', encoding='utf-8') as output:
        for filename in os.listdir(input_dir):
            if filename.endswith('.muc'):
                input_path = os.path.join(input_dir, filename)
                with open(input_path, 'r', encoding='utf-8') as file:
                    text = file.read()

                # Xử lý văn bản thành định dạng token-level
                processed_data = process_text(text)

                # Ghi từng token vào file output
                for token, ner in processed_data:
                    output.write(f"{token} {ner}\n")
                output.write("\n")  # Dòng trống giữa các file

if __name__ == "__main__":
    # Định nghĩa các thư mục input và file output
    input_output_mapping = {
        "/kaggle/input/ner-vlsp/VLSP2021/Dev-Muc": "dev-muc.txt",
        "/kaggle/input/ner-vlsp/VLSP2021/Train-Muc": "train.txt",
        "/kaggle/input/ner-vlsp/Testing/Test": "tests.txt"
    }

    # Xử lý từng thư mục
    for input_dir, output_file in input_output_mapping.items():
        print(f"Processing {input_dir} -> {output_file}")
        process_folder(input_dir, output_file)

    print("Hoàn thành xử lý tất cả các thư mục!")


Processing /kaggle/input/ner-vlsp/VLSP2021/Dev-Muc -> dev-muc.txt
Processing /kaggle/input/ner-vlsp/VLSP2021/Train-Muc -> train.txt
Processing /kaggle/input/ner-vlsp/Testing/Test -> tests.txt
Hoàn thành xử lý tất cả các thư mục!


In [4]:
from collections import defaultdict, Counter
from tqdm import tqdm

In [5]:
# Load train.txt để học tham số
def load_data(file_path):
    sentences, labels = [], []
    current_sentence, current_labels = [], []

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                token, label = line.strip().split()
                current_sentence.append(token)
                current_labels.append(label)
            else:
                sentences.append(current_sentence)
                labels.append(current_labels)
                current_sentence, current_labels = [], []

    return sentences, labels

In [6]:
def train_hmm(sentences, labels):
    """
    Học tham số HMM dựa trên token và NER tag.

    Args:
        sentences: Danh sách các câu, mỗi câu là một danh sách các token.
        labels: Danh sách các nhãn NER tương ứng với các câu.

    Returns:
        Tuple: Gồm các tham số của mô hình HMM:
            - initial_probs: Xác suất khởi đầu của các trạng thái.
            - transition_probs: Xác suất chuyển đổi giữa các trạng thái.
            - emission_probs: Xác suất phát xạ của các token từ các trạng thái.
            - states: Danh sách tất cả các trạng thái NER.
    """
    initial_probs = Counter()
    transition_probs = defaultdict(Counter)
    emission_probs = defaultdict(Counter)
    total_states = Counter()

    print("Bắt đầu huấn luyện mô hình HMM...")
    for sentence, label_seq in tqdm(zip(sentences, labels), total=len(sentences), desc="Đang huấn luyện"):
        initial_probs[label_seq[0]] += 1  # Đếm trạng thái khởi đầu
        for i in range(len(label_seq)):
            total_states[label_seq[i]] += 1  # Đếm số lần xuất hiện của mỗi trạng thái
            emission_probs[label_seq[i]][sentence[i]] += 1  # Đếm số lần token được phát xạ từ trạng thái
            if i > 0:
                transition_probs[label_seq[i - 1]][label_seq[i]] += 1  # Đếm số lần chuyển đổi giữa các trạng thái

    # Chuẩn hóa xác suất
    initial_probs = {k: v / sum(initial_probs.values()) for k, v in initial_probs.items()}
    transition_probs = {k: {k2: v2 / sum(v.values()) for k2, v2 in v.items()} for k, v in transition_probs.items()}
    emission_probs = {k: {k2: v2 / sum(v.values()) for k2, v2 in v.items()} for k, v in emission_probs.items()}

    print("Huấn luyện xong!")
    return initial_probs, transition_probs, emission_probs, list(total_states.keys())

In [7]:
def viterbi(sentence, initial_probs, transition_probs, emission_probs, states):
    """
    Thuật toán Viterbi để tìm chuỗi nhãn NER có xác suất cao nhất cho một câu.

    Args:
        sentence: Danh sách các token của câu cần dự đoán.
        initial_probs: Xác suất khởi đầu của các trạng thái.
        transition_probs: Xác suất chuyển đổi giữa các trạng thái.
        emission_probs: Xác suất phát xạ của các token từ các trạng thái.
        states: Danh sách tất cả các trạng thái NER.

    Returns:
        Danh sách các nhãn NER dự đoán cho câu.
    """
    dp = [{} for _ in range(len(sentence))]  # Khởi tạo bảng DP
    path = [{} for _ in range(len(sentence))]  # Khởi tạo bảng lưu trữ đường đi

    # Bước khởi tạo: Xác suất cho trạng thái đầu tiên
    for state in states:
        dp[0][state] = initial_probs.get(state, 0) * emission_probs[state].get(sentence[0], 1e-6)
        path[0][state] = [state]  # Lưu đường đi

    # Bước quy hoạch động: Tính xác suất cho các trạng thái tiếp theo
    for t in range(1, len(sentence)):
        for curr_state in states:
            max_prob, prev_state = max(
                (dp[t - 1][prev_state] * transition_probs[prev_state].get(curr_state, 1e-6) *
                 emission_probs[curr_state].get(sentence[t], 1e-6), prev_state)  # Tính xác suất chuyển đổi và phát xạ
                for prev_state in states
            )
            dp[t][curr_state] = max_prob  # Lưu xác suất cao nhất
            path[t][curr_state] = path[t - 1][prev_state] + [curr_state]  # Lưu đường đi

    # Tìm đường đi tốt nhất: Trạng thái cuối cùng có xác suất cao nhất
    final_state = max(dp[-1], key=dp[-1].get)

    # Trả về đường đi tốt nhất (chuỗi nhãn NER)
    return path[-1][final_state]

In [8]:
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score

def evaluate(dev_sentences, dev_labels, hmm_params):
    """
    Đánh giá hiệu suất của mô hình HMM trên tập development.

    Args:
        dev_sentences: Danh sách các câu trong tập development.
        dev_labels: Danh sách các nhãn NER tương ứng với các câu trong tập development.
        hmm_params: Các tham số của mô hình HMM (initial_probs, transition_probs, emission_probs, states).

    Returns:
        None. In ra kết quả đánh giá (precision, recall, F1-score).
    """
    initial_probs, transition_probs, emission_probs, states = hmm_params
    predictions = []

    print("\nĐang đánh giá mô hình HMM trên tập kiểm thử...")

    # Dự đoán nhãn cho các câu trong tập development
    for sentence in tqdm(dev_sentences, desc="Đang dự đoán"):
        pred = viterbi(sentence, initial_probs, transition_probs, emission_probs, states)
        predictions.append(pred)

    # Nối tất cả các nhãn và dự đoán
    y_true = [label for sublist in dev_labels for label in sublist]
    y_pred = [label for sublist in predictions for label in sublist]

    # Tính toán accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print("\nAccuracy:")
    print(f"{accuracy:.4f}")

    # Tính toán classification report (bao gồm micro, macro, weighted avg)
    print("\nClassification Report:")
    report = classification_report(y_true, y_pred, zero_division=0)
    print(report)

    # Tính toán thủ công micro avg, macro avg, và weighted avg nếu cần thêm chi tiết
    precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='micro', zero_division=0
    )
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0
    )
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted', zero_division=0
    )

    # In thêm thông tin chi tiết nếu cần
    print("\nChi tiết kết quả trung bình:")
    print(f"Micro avg - Precision: {precision_micro:.4f}, Recall: {recall_micro:.4f}, F1-Score: {f1_micro:.4f}")
    print(f"Macro avg - Precision: {precision_macro:.4f}, Recall: {recall_macro:.4f}, F1-Score: {f1_macro:.4f}")
    print(f"Weighted avg - Precision: {precision_weighted:.4f}, Recall: {recall_weighted:.4f}, F1-Score: {f1_weighted:.4f}")


In [9]:
# Main
train_file = '/kaggle/working/train.txt'
dev_file = '/kaggle/working/dev-muc.txt'
test_file = '/kaggle/working/tests.txt'

In [10]:
train_sentences, train_labels = load_data(train_file)
dev_sentences, dev_labels = load_data(dev_file)

In [11]:
type(train_sentences)

list

In [ ]:
train_sentences[0]

In [13]:
print(len(train_sentences))

1050


In [ ]:
train_sentences[1]

In [15]:
hmm_params = train_hmm(train_sentences, train_labels)

Bắt đầu huấn luyện mô hình HMM...


Đang huấn luyện: 100%|██████████| 1050/1050 [00:00<00:00, 2152.45it/s]


Huấn luyện xong!


In [16]:
import os

def count_files_in_directory(directory_path):
  """
  Đếm số lượng file trong một thư mục cụ thể.

  Args:
    directory_path: Đường dẫn đến thư mục cần đếm file.

  Returns:
    Số lượng file trong thư mục.
  """
  file_count = 0
  for filename in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, filename)):
      file_count += 1
  return file_count

# Đường dẫn đến thư mục bạn muốn đếm file
directory_path = "/kaggle/input/ner-vlsp/VLSP2021/Train-Muc"

# Đếm số lượng file trong thư mục
number_of_files = count_files_in_directory(directory_path)

# In ra kết quả
print(f"Số lượng file trong thư mục {directory_path}: {number_of_files}")

Số lượng file trong thư mục /kaggle/input/ner-vlsp/VLSP2021/Train-Muc: 1050


In [17]:
evaluate(dev_sentences, dev_labels, hmm_params)


Đang đánh giá mô hình HMM trên tập kiểm thử...


Đang dự đoán: 100%|██████████| 450/450 [16:49<00:00,  2.24s/it]



Accuracy:
0.8718

Classification Report:
                       precision    recall  f1-score   support

            B-ADDRESS       0.00      0.00      0.00        30
           B-DATETIME       0.64      0.11      0.19      2305
      B-DATETIME-DATE       0.51      0.08      0.13       688
 B-DATETIME-DATERANGE       0.21      0.04      0.07       316
  B-DATETIME-DURATION       0.54      0.05      0.10       902
       B-DATETIME-SET       0.00      0.00      0.00        73
      B-DATETIME-TIME       0.46      0.15      0.22        89
 B-DATETIME-TIMERANGE       0.03      0.01      0.01       159
              B-EMAIL       0.00      0.00      0.00         1
              B-EVENT       0.23      0.12      0.16       516
          B-EVENT-CUL       0.30      0.09      0.13       129
     B-EVENT-GAMESHOW       0.00      0.00      0.00       154
      B-EVENT-NATURAL       0.96      0.44      0.60        57
        B-EVENT-SPORT       0.19      0.12      0.15        97
           B

In [18]:
# Load dữ liệu test
test_sentences, test_labels = load_data(test_file) 

# Đánh giá mô hình trên tập test
evaluate(test_sentences, test_labels, hmm_params)


Đang đánh giá mô hình HMM trên tập kiểm thử...


Đang dự đoán: 100%|██████████| 310/310 [09:16<00:00,  1.79s/it]



Accuracy:
0.8312

Classification Report:
                       precision    recall  f1-score   support

            B-ADDRESS       0.13      0.03      0.05       204
           B-DATETIME       0.45      0.10      0.16      1283
      B-DATETIME-DATE       0.60      0.13      0.21      1087
 B-DATETIME-DATERANGE       0.18      0.04      0.06       327
  B-DATETIME-DURATION       0.44      0.07      0.12       716
       B-DATETIME-SET       0.00      0.00      0.00        13
      B-DATETIME-TIME       0.19      0.05      0.09        73
 B-DATETIME-TIMERANGE       0.51      0.22      0.31       165
              B-EMAIL       0.00      0.00      0.00         2
              B-EVENT       0.39      0.06      0.10       756
          B-EVENT-CUL       0.00      0.00      0.00        34
     B-EVENT-GAMESHOW       0.52      0.06      0.11       248
      B-EVENT-NATURAL       0.67      0.08      0.15        24
        B-EVENT-SPORT       0.76      0.10      0.17       491
            

In [19]:
# In bảng xác suất chuyển trạng thái
initial_probs, transition_probs, emission_probs, states = hmm_params
for current_state, next_states in transition_probs.items():
    print(f"Xác suất chuyển trạng thái từ {current_state}:")
    for next_state, prob in next_states.items():
        print(f"  - Đến {next_state}: {prob:.4f}")  # In với 4 chữ số thập phân
    print()

Xác suất chuyển trạng thái từ O:
  - Đến O: 0.9280
  - Đến B-PERSON: 0.0123
  - Đến B-QUANTITY-NUM: 0.0053
  - Đến B-DATETIME: 0.0025
  - Đến B-PERSONTYPE: 0.0047
  - Đến B-ORGANIZATION: 0.0091
  - Đến B-QUANTITY: 0.0036
  - Đến B-LOCATION-GPE: 0.0087
  - Đến B-DATETIME-DATE: 0.0015
  - Đến I-LOCATION: 0.0001
  - Đến I-DATETIME-TIME: 0.0000
  - Đến B-LOCATION: 0.0064
  - Đến B-PRODUCT: 0.0024
  - Đến B-QUANTITY-CUR: 0.0009
  - Đến B-ORGANIZATION-SPORTS: 0.0024
  - Đến B-PRODUCT-COM: 0.0009
  - Đến B-LOCATION-STRUC: 0.0007
  - Đến B-EVENT-SPORT: 0.0006
  - Đến B-EVENT-CUL: 0.0003
  - Đến B-LOCATION-GEO: 0.0005
  - Đến B-EVENT: 0.0006
  - Đến B-DATETIME-DATERANGE: 0.0003
  - Đến B-DATETIME-DURATION: 0.0010
  - Đến B-QUANTITY-DIM: 0.0003
  - Đến B-URL: 0.0003
  - Đến B-MISCELLANEOUS: 0.0008
  - Đến B-QUANTITY-ORD: 0.0010
  - Đến B-DATETIME-TIMERANGE: 0.0004
  - Đến B-DATETIME-TIME: 0.0005
  - Đến I-QUANTITY-NUM: 0.0010
  - Đến B-EVENT-NATURAL: 0.0001
  - Đến I-QUANTITY-DIM: 0.0000
  - Đến

In [20]:
new_sentence = "Sofascore là hãng thống kê hàng đầu thế giới, do người Croatia sáng lập năm 2010 và có hàng chục triệu người dùng. Theo mô hình của hãng, Xuân Son nhận điểm 10 và dĩ nhiên là cầu thủ hay nhất trận. Vĩ Hào đứng thứ hai với 8,4 điểm, nhờ một bàn và một đường kiến tạo, còn Nguyễn Quang Hải nhận 8,2 điểm với một bàn."

In [21]:
# Tiền xử lý câu văn
processed_sentence = [token for token, _ in process_text(new_sentence)]

# Dự đoán nhãn
predicted_labels = viterbi(processed_sentence, *hmm_params)

# Hiển thị kết quả theo định dạng yêu cầu
max_length = max(len(token) for token in processed_sentence)  # Để căn chỉnh đẹp
print("===== BẢNG DỰ ĐOÁN NER CHO ĐOẠN VĂN =====")
for token, label in zip(processed_sentence, predicted_labels):
    print(f"{token:<{max_length}} {label}")


===== BẢNG DỰ ĐOÁN NER CHO ĐOẠN VĂN =====
Sofascore O
là        O
hãng      O
thống     O
kê        O
hàng      O
đầu       O
thế       O
giới,     O
do        O
người     O
Croatia   B-LOCATION-GPE
sáng      O
lập       O
năm       B-DATETIME
2010      B-DATETIME
và        O
có        O
hàng      O
chục      B-QUANTITY-CUR
triệu     B-QUANTITY-CUR
người     O
dùng.     O
Theo      O
mô        O
hình      O
của       O
hãng,     O
Xuân      B-PERSON
Son       B-PERSON
nhận      O
điểm      O
10        B-QUANTITY-NUM
và        O
dĩ        O
nhiên     O
là        O
cầu       O
thủ       O
hay       O
nhất      O
trận.     O
Vĩ        B-PERSON
Hào       B-PERSON
đứng      O
thứ       B-QUANTITY-ORD
hai       B-QUANTITY-ORD
với       O
8,4       O
điểm,     O
nhờ       O
một       O
bàn       O
và        O
một       O
đường     O
kiến      O
tạo,      O
còn       O
Nguyễn    B-PERSON
Quang     B-PERSON
Hải       B-PERSON
nhận      O
8,2       O
điểm      O
với       O
một       O
bàn.     

In [22]:
# Tiền xử lý câu văn
processed_sentence = [token for token, _ in process_text(new_sentence)]

# Dự đoán nhãn
predicted_labels = viterbi(processed_sentence, *hmm_params)

# Loại bỏ nhãn "O"
filtered_tokens_labels = [(token, label) for token, label in zip(processed_sentence, predicted_labels) if label != "O"]

# Hiển thị kết quả theo định dạng yêu cầu
print("----------------------------------------------------")
for token, label in filtered_tokens_labels:
    print(f"Token: {token} Label: {label}")
print("----------------------------------------------------")


----------------------------------------------------
Token: Croatia Label: B-LOCATION-GPE
Token: năm Label: B-DATETIME
Token: 2010 Label: B-DATETIME
Token: chục Label: B-QUANTITY-CUR
Token: triệu Label: B-QUANTITY-CUR
Token: Xuân Label: B-PERSON
Token: Son Label: B-PERSON
Token: 10 Label: B-QUANTITY-NUM
Token: Vĩ Label: B-PERSON
Token: Hào Label: B-PERSON
Token: thứ Label: B-QUANTITY-ORD
Token: hai Label: B-QUANTITY-ORD
Token: Nguyễn Label: B-PERSON
Token: Quang Label: B-PERSON
Token: Hải Label: B-PERSON
----------------------------------------------------
